In [3]:
import cv2
import numpy as np
import os
import pandas as pd
import csv

from sklearn.cluster import MiniBatchKMeans
# from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from rembg import remove
import glob
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import metrics
from commonFunctions import *

## Extract keypoints from each image

In [ ]:
img_path = '../input/images/'
train = pd.read_csv('../input/train.csv')
species = train.species.sort_values().unique()

dico = []

def step1():
    for leaf in train.id:
        img = cv2.imread(img_path + str(leaf) + ".jpg")
        kp, des = cv2.sift.detectAndCompute(img, None)

        for d in des:
            dico.append(d)

In [4]:
arr = ["0","1","2","3","4","5"]
dico = []
lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (256,256))
            img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
            mask = cv2.inRange(img_hls, lower_blue, upper_blue)
            result = cv2.bitwise_and(img, img, mask=mask)
            
            
            sift = cv2.ORB_create()
            kp, des = sift.detectAndCompute(result, None)
            for d in des:
                dico.append(d)
        except:
            pass
        

In [1]:
import numpy as np
import cv2

# Load image and detect SIFT keypoints and descriptors
gray = cv2.imread('test.png', 0)
sift = cv2.ORB_create()
kp, des = sift.detectAndCompute(gray, None)

# Stack descriptors into a NumPy array
# des_array = np.vstack(des)
dd = []
for d in des:
    dd.append(d)
# print(np.shape(des_array))
print(np.shape(dd))


(500, 32)


In [35]:
img = cv2.imread('test.png')
gray= cv2.cvtColor(img,cv.COLOR_BGR2GRAY)
sift = cv2.SIFT_create()
kp = sift.detect(gray,None)
des = sift.compute(gray,kp)
des = np.array(des)

# desc_samples = des[np. random. randint(len(des), size = 20)]
# print(desc_samples)
print(np.shape(des))
img=cv2.drawKeypoints(gray,kp,img)
# cv2.imwrite('sift_keypoints.jpg',img)
# show_images([img])

C:\Users\khale\AppData\Local\Temp\ipykernel_3948\479942303.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  des = np.array(des)


ValueError: could not broadcast input array from shape (608,128) into shape (608,)

## Clustering
We now have an array with a huge number of descriptors. We cannot use all of them to create or model so we need to cluster them. A rule-of-thumb is to create k centers with k = number of categories * 10 (in our case, it's 60).

In [ ]:
def step2():
    k = np.size(species) * 10

    batch_size = np.size(os.listdir(img_path)) * 3
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)
    
# I use MiniBatchKMeans to avoid Memory Error.

In [5]:
print(np.shape(dico))

(398610, 32)


In [1]:
# print(np.shape(dico[0]))
# print(np.shape(dico[1]))
# print(np.shape(dico[2]))
# print(np.shape(dico[3]))
# print(np.shape(dico[4]))
# print(np.shape(dico[5]))
# print(np.shape(dico[6]))
# print(np.shape(dico[7]))
# print(np.shape(dico[8]))
# print(np.shape(dico[9]))

# dico2 = dico[1::2]
# for i in range(len(dico2)):
#     print(np.shape(dico2[i]))


In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=60)
kmeans.fit(dico)
joblib.dump(kmeans, "model/Tkmeans.pkl")

c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


['model/Tkmeans.pkl']

In [6]:
k = 10 * 6 
batch_size = 50
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)
# kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1, max_iter=500).fit(dico)

joblib.dump(kmeans, "model/Bkmeans.pkl")

c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Init 1/3 with method k-means++
Inertia for init 1/3: 20149374.0
Init 2/3 with method k-means++
Inertia for init 2/3: 19587426.0
Init 3/3 with method k-means++
Inertia for init 3/3: 19913363.0
[MiniBatchKMeans] Reassigning 25 cluster centers.
Minibatch step 1/797220: mean batch inertia: 137874.16
[MiniBatchKMeans] Reassigning 24 cluster centers.
Minibatch step 2/797220: mean batch inertia: 136007.66477777777, ewa inertia: 136007.66477777777
Minibatch step 3/797220: mean batch inertia: 115860.1145442914, ewa inertia: 136002.61033867963
Minibatch step 4/797220: mean batch inertia: 119991.44674643509, ewa inertia: 135998.59359965532
Minibatch step 5/797220: mean batch inertia: 115373.2097047511, ewa inertia: 135993.419285877
Minibatch step 6/797220: mean batch inertia: 116491.73227694449, ewa inertia: 135988.5268752288
Minibatch step 7/797220: mean batch inertia: 112425.59558575787, ewa inertia: 135982.6156155572
Minibatch step 8/797220: mean batch inertia: 101358.29748651212, ewa inertia:

['model/Bkmeans.pkl']

## Creation of the histograms
To create our each image by a histogram. We will create a vector of k value for each image. For each keypoints in an image, we will find the nearest center and increase by one its value.

In [ ]:
def step3():
    kmeans.verbose = False

    histo_list = []

    for leaf in train.id:
        img = cv2.imread(img_path + str(leaf) + ".jpg")
        kp, des = sift.detectAndCompute(img, None)

        histo = np.zeros(k)
        nkp = np.size(kp)

        for d in des:
            idx = kmeans.predict([d])
            histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

        histo_list.append(histo)

In [7]:
# kmeans.verbose = False
# kmeans=joblib.load("model/kmeans.pkl")
histo_list = []
Y = []
arr = ["0","1","2","3","4","5"]

lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (256,256))
            img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
            mask = cv2.inRange(img_hls, lower_blue, upper_blue)
            result = cv2.bitwise_and(img, img, mask=mask)
            
            
            sift = cv2.ORB_create()
            kp, des = sift.detectAndCompute(result, None)
            histo = np.zeros(60)
            nkp = np.size(kp)
            for d in des:
                idx = kmeans.predict([d])
                histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

            histo_list.append(histo)
            Y.append(int(label))
        except:
            pass
        

## train our model

In [12]:
print(Y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [8]:
print(np.shape(histo_list))
Y = [int(num) for num in Y]

(1047, 60)


In [9]:
X = np.array(histo_list)
train_x,test_x,train_y,test_y = train_test_split(histo_list,Y, test_size = 0.3)
classifier=SVC(kernel="linear", random_state=6)
classifier.fit(train_x,train_y)
joblib.dump(classifier, "model/Bsift_model.pkl")

['model/Bsift_model.pkl']

In [10]:
from sklearn import metrics

prediction=classifier.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.2507936507936508
Precision=  0.8318875024757378
Recall=  0.2507936507936508
F1 Score=  0.38535677711944993
Confusion Matrix=  [[ 0  0  0  0  0  0]
 [48 43 35 23 16  8]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [17  8 17 34 30 36]]
Classification Report=                precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.84      0.25      0.38       173
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.82      0.25      0.39       142

    accuracy                           0.25       315
   macro avg       0.28      0.08      0.13       315
weighted avg       0.83      0.25      0.39       315



c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier and fit it to the training data
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(train_x, train_y)


RandomForestClassifier()

In [14]:
prediction=rfc.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.45396825396825397
Precision=  0.5094645547788635
Recall=  0.45396825396825397
F1 Score=  0.47086838262949077
Confusion Matrix=  [[38  4  7  8  5  1]
 [11 30 19  5  3  2]
 [ 8  8 12  8  4  0]
 [ 4  3  6 13  6  3]
 [ 3  4  5  9 16  4]
 [ 1  2  3 14 12 34]]
Classification Report=                precision    recall  f1-score   support

           0       0.58      0.60      0.59        63
           1       0.59      0.43      0.50        70
           2       0.23      0.30      0.26        40
           3       0.23      0.37      0.28        35
           4       0.35      0.39      0.37        41
           5       0.77      0.52      0.62        66

    accuracy                           0.45       315
   macro avg       0.46      0.43      0.44       315
weighted avg       0.51      0.45      0.47       315



In [15]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100)
gbc.fit(train_x, train_y)

GradientBoostingClassifier()

In [16]:
prediction=gbc.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.42857142857142855
Precision=  0.44199665104093694
Recall=  0.42857142857142855
F1 Score=  0.431878059160172
Confusion Matrix=  [[36  4  4  6  3  1]
 [12 23 16  4  6  0]
 [ 8 11 15 13  2  4]
 [ 3  6  9 12  9  4]
 [ 4  6  4 12 19  5]
 [ 2  1  4 10  7 30]]
Classification Report=                precision    recall  f1-score   support

           0       0.55      0.67      0.61        54
           1       0.45      0.38      0.41        61
           2       0.29      0.28      0.29        53
           3       0.21      0.28      0.24        43
           4       0.41      0.38      0.40        50
           5       0.68      0.56      0.61        54

    accuracy                           0.43       315
   macro avg       0.43      0.42      0.42       315
weighted avg       0.44      0.43      0.43       315



In [17]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_x, train_y)

KNeighborsClassifier()

In [18]:
prediction=knn.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.35555555555555557
Precision=  0.3523405611304386
Recall=  0.35555555555555557
F1 Score=  0.3513510963012624
Confusion Matrix=  [[35  7  9  9  4  3]
 [12 15 16  6  4  0]
 [ 7 14  9 10 10  6]
 [ 8 12 13 17 16  6]
 [ 2  2  3 13  9  2]
 [ 1  1  2  2  3 27]]
Classification Report=                precision    recall  f1-score   support

           0       0.54      0.52      0.53        67
           1       0.29      0.28      0.29        53
           2       0.17      0.16      0.17        56
           3       0.30      0.24      0.26        72
           4       0.20      0.29      0.23        31
           5       0.61      0.75      0.67        36

    accuracy                           0.36       315
   macro avg       0.35      0.37      0.36       315
weighted avg       0.35      0.36      0.35       315



In [19]:
from sklearn.ensemble import AdaBoostClassifier

adab_clf = AdaBoostClassifier(random_state=42)
adab_clf.fit(train_x, train_y)

AdaBoostClassifier(random_state=42)

In [20]:
prediction=adab_clf.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.34285714285714286
Precision=  0.34302671882292285
Recall=  0.34285714285714286
F1 Score=  0.342274470316046
Confusion Matrix=  [[29  9  9  8  4  2]
 [ 4 20 16  6  9  4]
 [11 10 10  8  4  4]
 [11  6  6 18 11  2]
 [ 9  4  7 10 10 11]
 [ 1  2  4  7  8 21]]
Classification Report=                precision    recall  f1-score   support

           0       0.45      0.48      0.46        61
           1       0.39      0.34      0.36        59
           2       0.19      0.21      0.20        47
           3       0.32      0.33      0.32        54
           4       0.22      0.20      0.21        51
           5       0.48      0.49      0.48        43

    accuracy                           0.34       315
   macro avg       0.34      0.34      0.34       315
weighted avg       0.34      0.34      0.34       315



In [21]:
import xgboost as xgb
# Train an XGBoost Classifier on the training set
dtrain = xgb.DMatrix(train_x, label=train_y)
dtest = xgb.DMatrix(test_x, label=test_y)

param = {'max_depth': 3, 'eta': 0.1, 'objective': 'multi:softmax', 'num_class': 6}
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [23]:
prediction=bst.predict(dtest)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.40634920634920635
Precision=  0.4390410308595798
Recall=  0.40634920634920635
F1 Score=  0.4125413416154208
Confusion Matrix=  [[33  3  3  8  2  0]
 [ 8 22 13  4  6  5]
 [ 5 11 13  8  2  2]
 [ 9  4  6 12  8  3]
 [ 5  8  9 14 17  3]
 [ 5  3  8 11 11 31]]
Classification Report=                precision    recall  f1-score   support

         0.0       0.51      0.67      0.58        49
         1.0       0.43      0.38      0.40        58
         2.0       0.25      0.32      0.28        41
         3.0       0.21      0.29      0.24        42
         4.0       0.37      0.30      0.33        56
         5.0       0.70      0.45      0.55        69

    accuracy                           0.41       315
   macro avg       0.41      0.40      0.40       315
weighted avg       0.44      0.41      0.41       315



In [ ]:
def step4():
    X = np.array(histo_list)
    Y = []

    # It's a way to convert species name into an integer
    for s in train.species:
        Y.append(np.min(np.nonzero(species == s)))

    mlp = MLPClassifier(verbose=True, max_iter=600000)
    mlp.fit(X, Y)

I also run this script with ORB instead of SIFT and I got best results. To do it, just replace `cv2.xfeatures2d.SIFT_create()` by `cv2.ORB_create().`